In [66]:
%env SPARK_VERSION=3.0.0

env: SPARK_VERSION=3.0.0


In [67]:
import pydeequ

from pyspark.sql import SparkSession, Row

# TODO create spark session with jdbc driver path

In [68]:
server_name = "jdbc:sqlserver://host.docker.internal"
database_name = "TRN"
jdbc_url = server_name + ";" + "databaseName=" + database_name + ";trustServerCertificate=True;"

table_name = "hr.jobs"
username = "DQTestUser"
password = "DQTesting111" # Please specify password here
connection_details = { "user": username, "password": password, "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver", }


spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.extraClassPath", "/home/jars/sqlserver/sqlserverjdbc.jar")
    .getOrCreate())

df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_details)

In [69]:
df.printSchema()

root
 |-- job_id: integer (nullable = true)
 |-- job_title: string (nullable = true)
 |-- min_salary: decimal(8,2) (nullable = true)
 |-- max_salary: decimal(8,2) (nullable = true)



In [70]:
df.show(20,False)

+------+-------------------------------+----------+----------+
|job_id|job_title                      |min_salary|max_salary|
+------+-------------------------------+----------+----------+
|1     |Public Accountant              |4200.00   |9000.00   |
|2     |Accounting Manager             |8200.00   |16000.00  |
|3     |Administration Assistant       |3000.00   |6000.00   |
|4     |President                      |20000.00  |40000.00  |
|5     |Administration Vice President  |15000.00  |30000.00  |
|6     |Accountant                     |4200.00   |9000.00   |
|7     |Finance Manager                |8200.00   |16000.00  |
|8     |Human Resources Representative |4000.00   |9000.00   |
|9     |Programmer                     |4000.00   |10000.00  |
|10    |Marketing Manager              |9000.00   |15000.00  |
|11    |Marketing Representative       |4000.00   |9000.00   |
|12    |Public Relations Representative|4500.00   |10500.00  |
|13    |Purchasing Clerk               |2500.00   |5500

In [71]:
### Data Analyzers section
# TODO analyze data here
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(ApproxCountDistinct("job_title")) \
                    .addAnalyzer(Completeness("job_id")) \
                    .addAnalyzer(ApproxCountDistinct("job_id")) \
                    .addAnalyzer(Mean("min_salary")) \
                    .addAnalyzer(Mean("max_salary")) \
                    .addAnalyzer(Compliance("min_salary", "min_salary > 0")) \
                    .addAnalyzer(Compliance("max_salary", "max_salary > 0")) \
                    .run()
                    #.addAnalyzer(Compliance("top star_rating", "star_rating >= 4.0")) \
                    #.addAnalyzer(Correlation("total_votes", "star_rating")) \
                    #.addAnalyzer(Correlation("total_votes", "helpful_votes")) \
                    
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+----------+-------------------+------------------+
| entity|  instance|               name|             value|
+-------+----------+-------------------+------------------+
| Column|    job_id|       Completeness|               1.0|
| Column|    job_id|ApproxCountDistinct|              19.0|
| Column|max_salary|         Compliance|               0.0|
| Column| job_title|ApproxCountDistinct|              18.0|
| Column|min_salary|               Mean| 6568.421052631579|
|Dataset|         *|               Size|              19.0|
| Column|max_salary|               Mean|13210.526315789473|
| Column|min_salary|         Compliance|               1.0|
+-------+----------+-------------------+------------------+



In [30]:
analysisResult_pandas_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult, pandas=True)
analysisResult_pandas_df

,entity,instance,name,value
0,Column,job_id,Completeness,1.000000
1,Column,job_id,ApproxCountDistinct,19.000000
2,Column,max_salary,Compliance,0.000000
3,Column,job_title,ApproxCountDistinct,18.000000
4,Column,min_salary,Mean,6568.421053
5,Dataset,*,Size,19.000000
6,Column,max_salary,Mean,13210.526316
7,Column,min_salary,Compliance,1.000000


In [80]:
### Data profiling section
# TODO profile data here
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print("Column '{}':\n ".format(col) +
        "\tcompleteness: {}\n".format(profile.completeness) +
        "\tapproximate number of distinct values: {}\n".format(profile.approximateNumDistinctValues) +
        "\tdatatype: {}\n".format(profile.dataType) +
        ("\tmean: {}\n".format(profile.mean) if profile.dataType == 'Integral' else '') +
        ("\tmax: {}\n".format(profile.maximum) if profile.dataType == 'Integral' else '' ) +
        ("\tmin: {}\n".format(profile.minimum) if profile.dataType == 'Integral' else '' ) +
        ("\tsum: {}\n".format(profile.sum) if profile.dataType == 'Integral' else '' )
         )

print('Value distribution:\r\n')
for col, profile in result.profiles.items():
    if profile.histogram is not None:
        print('Column {}'.format(col))
        for each in profile.histogram:
            print('\t\tValue {} occured {} times (ratio is {})'.format(each.value, each.count,round(each.ratio,2)))

Column 'job_id':
 	completeness: 1.0
	approximate number of distinct values: 19
	datatype: Integral
	mean: 10.0
	max: 19.0
	min: 1.0
	sum: 190.0

Column 'job_title':
 	completeness: 1.0
	approximate number of distinct values: 18
	datatype: String

Column 'min_salary':
 	completeness: 1.0
	approximate number of distinct values: 14
	datatype: Fractional
	mean: 6568.421052631579
	max: 20000.0
	min: 2000.0
	sum: 124800.0

Column 'max_salary':
 	completeness: 1.0
	approximate number of distinct values: 13
	datatype: Fractional
	mean: 13210.526315789473
	max: 40000.0
	min: 5000.0
	sum: 251000.0

Value distribution:

Column job_id
		Value 12 occured 1 times (ratio is 0.05)
		Value 8 occured 1 times (ratio is 0.05)
		Value 19 occured 1 times (ratio is 0.05)
		Value 4 occured 1 times (ratio is 0.05)
		Value 15 occured 1 times (ratio is 0.05)
		Value 11 occured 1 times (ratio is 0.05)
		Value 9 occured 1 times (ratio is 0.05)
		Value 13 occured 1 times (ratio is 0.05)
		Value 16 occured 1 times 

In [63]:
### Constraint Suggestions section
# TODO find meaninful constraints here
from pydeequ.suggestions import *
from pyspark.sql import Row

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
#print(suggestionResult)
suggdf = spark.createDataFrame(Row(**x) for x in suggestionResult['constraint_suggestions']) #.show(truncate=False)
suggdf.sort("column_name","current_value").select('column_name','current_value','description','code_for_constraint').show(truncate=False)

+-----------+--------------------------------------+-----------------------------------+----------------------------+
|column_name|current_value                         |description                        |code_for_constraint         |
+-----------+--------------------------------------+-----------------------------------+----------------------------+
|job_id     |ApproxDistinctness: 1.0               |'job_id' is unique                 |.isUnique("job_id")         |
|job_id     |Completeness: 1.0                     |'job_id' is not null               |.isComplete("job_id")       |
|job_id     |Minimum: 1.0                          |'job_id' has no negative values    |.isNonNegative("job_id")    |
|job_title  |ApproxDistinctness: 0.9473684210526315|'job_title' is unique              |.isUnique("job_title")      |
|job_title  |Completeness: 1.0                     |'job_title' is not null            |.isComplete("job_title")    |
|max_salary |Completeness: 1.0                     |'max

In [64]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check \
        .hasMin("job_id", lambda x: x == 1) \
        .isComplete("job_title")  \
        .isComplete("max_salary") \
        .isComplete("min_salary") \
        .isUnique("job_id")  \
        .isNonNegative("max_salary") \
        .isNonNegative("min_salary")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|check       |check_level|check_status|constraint                                                                                                              |constraint_status|constraint_message|
+------------+-----------+------------+------------------------------------------------------------------------------------------------------------------------+-----------------+------------------+
|Review Check|Warning    |Success     |MinimumConstraint(Minimum(job_id,None))                                                                                 |Success          |                  |
|Review Check|Warning    |Success     |CompletenessConstraint(Completeness(job_title,None))                                                                    |Success          |                  |
|Review Ch

In [65]:
checkResult_pandas_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas=True)
checkResult_pandas_df

,entity,instance,name,value
0,Column,job_id,Minimum,1.0
1,Column,job_title,Completeness,1.0
2,Column,min_salary,Completeness,1.0
3,Column,job_id,Uniqueness,1.0
4,Column,min_salary is non-negative,Compliance,1.0
5,Column,max_salary,Completeness,1.0
6,Column,max_salary is non-negative,Compliance,1.0
